In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import dataframe_image as dfi
from datetime import datetime
import sklearn.metrics as mtr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Função que cria os Modelos

In [2]:
def cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino):

    #Treino do Modelo XGB e salvamento do tempo
    inicio_tempo_treino_xgb = datetime.now()
    modelo_xgb = xgb.XGBRegressor(gamma = 0.0,
    learning_rate = 0.01,
    max_depth = 4,
    min_child_weight = 8,
    n_estimators = 500)
    modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])

    fim_tempo_treino_xgb = datetime.now()
    tempo_treino_xgb = fim_tempo_treino_xgb - inicio_tempo_treino_xgb
    tempo_treino_xgb = float(f"{str(tempo_treino_xgb.seconds)}.{str(tempo_treino_xgb.microseconds)}")

    #Treino do Modelo RF e salvamento do tempo
    inicio_tempo_treino_rf = datetime.now()
    modelo_rf = RandomForestRegressor(bootstrap = True,
    max_depth = 11,
    max_features = 'log2',
    min_samples_leaf = 7,
    min_samples_split = 6,
    n_estimators = 500)
    modelo_rf.fit(x_treino, y_treino)

    fim_tempo_treino_rf = datetime.now()
    tempo_treino_rf = fim_tempo_treino_rf - inicio_tempo_treino_rf
    tempo_treino_rf = float(f"{str(tempo_treino_rf.seconds)}.{str(tempo_treino_rf.microseconds)}")

    #Treino do Modelo LR e salvamento do tempo
    inicio_tempo_treino_lr = datetime.now()
    modelo_lr = LinearRegression()
    modelo_lr.fit(x_treino, y_treino)

    fim_tempo_treino_lr = datetime.now()
    tempo_treino_lr = fim_tempo_treino_lr - inicio_tempo_treino_lr
    tempo_treino_lr = float(f"{str(tempo_treino_lr.seconds)}.{str(tempo_treino_lr.microseconds)}")

    #Nomeando a coluna que receberá as previsões
    TesteNomeXGB = f"XGB - {TesteNome}"
    TesteNomeRF = f"RF - {TesteNome}"
    TesteNomeLR = f"LR - {TesteNome}"

    #Predição do XGB e salvamento do tempo
    inicio_tempo_previsao_xgb = datetime.now()
    df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
    fim_tempo_previsao_xgb = datetime.now()
    tempo_previsao_xgb = fim_tempo_previsao_xgb - inicio_tempo_previsao_xgb
    tempo_previsao_xgb = float(f"{str(tempo_previsao_xgb.seconds)}.{str(tempo_previsao_xgb.microseconds)}")

    #Predição do RF e salvamento do tempo
    inicio_tempo_previsao_rf = datetime.now()
    df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
    fim_tempo_previsao_rf = datetime.now()
    tempo_previsao_rf = fim_tempo_previsao_rf - inicio_tempo_previsao_rf
    tempo_previsao_rf = float(f"{str(tempo_previsao_rf.seconds)}.{str(tempo_previsao_rf.microseconds)}")

    #Predição do LR e salvamento do tempo
    inicio_tempo_previsao_lr = datetime.now()
    df_teste[f'{TesteNomeLR}'] = modelo_lr.predict(x_teste)
    fim_tempo_previsao_lr = datetime.now()
    tempo_previsao_lr = fim_tempo_previsao_lr - inicio_tempo_previsao_lr
    tempo_previsao_lr = float(f"{str(tempo_previsao_lr.seconds)}.{str(tempo_previsao_lr.microseconds)}")

    #Métricas do XGB
    RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeXGB}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeXGB}"])

    #Métricas do RF
    RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeRF}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeRF}"])

    #Métricas do LR
    RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeLR}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeLR}"])

    #Salvando as métricas num DataFrame Bonitinho
    metrics.append([f'{TesteNome}', Rota,'XGBoost','RRSE',RRSE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'XGBoost','MAPE',MAPE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','RRSE',RRSE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','MAPE',MAPE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','RRSE',RRSE_lr, tempo_treino_lr, tempo_previsao_lr])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','MAPE',MAPE_lr, tempo_treino_lr, tempo_previsao_lr])

    #Salvando as predições num DataFrame Bonitinho
    for i in range(df_teste.shape[0]):
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'XGBoost',df_teste[f"{TesteNomeXGB}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Random Forest',df_teste[f"{TesteNomeRF}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Linear Regression',df_teste[f"{TesteNomeLR}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Tempo real',df_teste["tempo_viagem"].iloc[i]])

    # fi_xgb = pd.DataFrame(data=modelo_xgb.feature_importances_, index=modelo_xgb.feature_names_in_, columns=["importance"])
    # fi_xgb.sort_values("importance").plot(kind="barh", title="Importância Dados XGB")
    # plt.savefig(f"Imagens/Feature Importance/FI_XGB_{TesteNome}.jpg")

    # fi_rf = pd.DataFrame(data=modelo_rf.feature_importances_, index=x_treino.columns, columns=["importance"])
    # fi_rf.sort_values("importance").plot(kind="barh", title="Importância Dados RF")
    # plt.savefig(f"Imagens/Feature Importance/FI_RF_{TesteNome}.jpg")

    # coefs_lr = pd.DataFrame(modelo_lr.coef_, columns=['Coeficiente'], index = x_treino.columns)
    # coefs_lr.Coeficiente = coefs_lr.Coeficiente.abs()
    # coefs_lr['DesvioPadrão'] = x_treino.std(axis=0)
    # coefs_lr['Importancia'] = (100*coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'])/coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'].max()
    # coefs_lr['Importancia'].plot(kind='barh', title = 'Importância Dados LR')
    # plt.savefig(f"Imagens/Feature Importance/FI_LR_{TesteNome}.jpg")
    

# Parte pra editar os teste

In [10]:
df = pd.read_csv('Clean/rota410FL.csv')
df.head()
Rota = "FL - 410"

In [11]:
def outlier_metric(valores):
    Q1 = np.percentile(valores,25)
    Q3 = np.percentile(valores,75)
    intervalo = Q3-Q1
    limite_superior = Q3+intervalo*1.5
    limite_inferior = Q1-intervalo*1.5
    return limite_superior,limite_inferior
limite_superior,limite_inferior = outlier_metric(df['tempo_viagem'])
df = df.loc[(df['tempo_viagem'] < limite_superior) & (df['tempo_viagem'] > limite_inferior)]

In [12]:
predictions = []
metrics = []

In [14]:
metrics = pd.DataFrame(metrics, columns=['Teste','Rota','Modelo','Métrica','Valor', 'Tempo de treino', 'Tempo de previsão'])
predictions = pd.DataFrame(predictions, columns=['Teste','Data','Rota','Modelo','Valor'])
# metrics.to_csv('metrics.csv', index = False)
# predictions.to_csv('predictions.csv', index = False)

In [15]:
metrics

,Teste,Rota,Modelo,Métrica,Valor,Tempo de treino,Tempo de previsão
0,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 410,XGBoost,RRSE,0.899073,2.135000,0.599800
1,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 410,XGBoost,MAPE,0.198408,2.135000,0.599800
2,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 410,Random Forest,RRSE,0.944451,2.173031,0.117031
3,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 410,Random Forest,MAPE,0.222702,2.173031,0.117031
4,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 410,Linear Regression,RRSE,0.973450,0.297000,0.971000
...,...,...,...,...,...,...,...
133,4 Teste de quantidade de dados para treino - 1...,FL - 410,XGBoost,MAPE,0.192577,1.323403,0.499800
134,4 Teste de quantidade de dados para treino - 1...,FL - 410,Random Forest,RRSE,0.873052,2.261002,0.197030
135,4 Teste de quantidade de dados para treino - 1...,FL - 410,Random Forest,MAPE,0.193463,2.261002,0.197030
136,4 Teste de quantidade de dados para treino - 1...,FL - 410,Linear Regression,RRSE,0.987499,0.300000,0.200000


In [5]:
Colunas = [
    #Primeira parte teste de colunas
    ["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia"],
    ["dia_semana", "tipo_dia", "turno_dia"],
    ["dia_semana", "tipo_dia", "hora_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia","partidaTimeStamp"],
    ["dia_semana", "tipo_dia", "turno_dia","partidaTimeStamp"],

    #Segunda parte teste com dados de clima
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva", "Calor"],
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao"],
    ["dia_semana", "tipo_dia", "hora_dia", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva"],
    ["dia_semana", "tipo_dia", "hora_dia", "Calor"],

    #Terceira parte teste com os tempo de viagens anteriores
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "tempo_viagem_5"],

    #Quarta parte alternar percentual de treino
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]
    
]

TestesNome = [
    #Primeira parte teste de colunas
    "0 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, turno_dia, qtdDiasAno, partidaTimeStamp]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, qtdDiasAno]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, qtdDiasAno]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, partidaTimeStamp]",
    "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, partidaTimeStamp]",

    #Segunda parte teste com dados de clima
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao, Temperatura]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva, Calor]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Temperatura]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva]",
    "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Calor]",

    #Terceira parte teste com os tempo de viagens anteriores
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4]",
    "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4, tempo_viagem_5]",

    #Quarta parte alternar percentual de treino
    "4 Teste de quantidade de dados para treino - 2 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 4 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 6 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 8 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    "4 Teste de quantidade de dados para treino - 10 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]"

]

PercentTeste = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.152173, 0.304347, 0.456521, 0.608695, 0.760869]


In [6]:
print(f'{len(Colunas)} --- {len(TestesNome)} --- {len(PercentTeste)}')

23 --- 23 --- 23


In [13]:
#for i in range(9,10,1):
for i in range(len(TestesNome)):
    shape = int(df.shape[0] * (1-PercentTeste[i]))
    df = df.sort_values(by=["data_partida"], ascending=False)
    df_teste = df[:shape]
    df_treino = df[shape:]

    TesteNome = TestesNome[i]

    x_teste = df_teste[Colunas[i]]
    y_teste = df_teste["tempo_viagem"]
    x_treino = df_treino[Colunas[i]]
    y_treino = df_treino["tempo_viagem"]

    cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino)

[0]	validation_0-rmse:1710.61075
[1]	validation_0-rmse:1694.39965
[2]	validation_0-rmse:1678.35909
[3]	validation_0-rmse:1662.48734
[4]	validation_0-rmse:1646.78276
[5]	validation_0-rmse:1631.24381
[6]	validation_0-rmse:1615.86886
[7]	validation_0-rmse:1600.65641
[8]	validation_0-rmse:1585.60480
[9]	validation_0-rmse:1570.71263
[10]	validation_0-rmse:1555.97825
[11]	validation_0-rmse:1541.40021
[12]	validation_0-rmse:1526.97710
[13]	validation_0-rmse:1512.70726
[14]	validation_0-rmse:1498.58943
[15]	validation_0-rmse:1484.62203
[16]	validation_0-rmse:1470.80364
[17]	validation_0-rmse:1457.13297
[18]	validation_0-rmse:1443.60845
[19]	validation_0-rmse:1430.22885
[20]	validation_0-rmse:1416.99271
[21]	validation_0-rmse:1403.89870
[22]	validation_0-rmse:1390.94540
[23]	validation_0-rmse:1378.13163
[24]	validation_0-rmse:1365.45601
[25]	validation_0-rmse:1352.91711
[26]	validation_0-rmse:1340.51382
[27]	validation_0-rmse:1328.24479
[28]	validation_0-rmse:1316.10880
[29]	validation_0-rmse:1

C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1710.61100
[1]	validation_0-rmse:1694.40019
[2]	validation_0-rmse:1678.35992
[3]	validation_0-rmse:1662.48849
[4]	validation_0-rmse:1646.78426
[5]	validation_0-rmse:1631.24568
[6]	validation_0-rmse:1615.87113
[7]	validation_0-rmse:1600.65909
[8]	validation_0-rmse:1585.60792
[9]	validation_0-rmse:1570.71621
[10]	validation_0-rmse:1555.98232
[11]	validation_0-rmse:1541.40482
[12]	validation_0-rmse:1526.98226
[13]	validation_0-rmse:1512.71302
[14]	validation_0-rmse:1498.59581
[15]	validation_0-rmse:1484.62907
[16]	validation_0-rmse:1470.81138
[17]	validation_0-rmse:1457.14142
[18]	validation_0-rmse:1443.61768
[19]	validation_0-rmse:1430.23886
[20]	validation_0-rmse:1417.00354
[21]	validation_0-rmse:1403.91038
[22]	validation_0-rmse:1390.95797
[23]	validation_0-rmse:1378.14512
[24]	validation_0-rmse:1365.47044
[25]	validation_0-rmse:1352.93253
[26]	validation_0-rmse:1340.53025
[27]	validation_0-rmse:1328.26226
[28]	validation_0-rmse:1316.12733
[29]	validation_0-rmse:1

C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1710.76918
[1]	validation_0-rmse:1694.71653
[2]	validation_0-rmse:1678.83425
[3]	validation_0-rmse:1663.12072
[4]	validation_0-rmse:1647.57440
[5]	validation_0-rmse:1632.19362
[6]	validation_0-rmse:1616.97689
[7]	validation_0-rmse:1601.92257
[8]	validation_0-rmse:1587.02916
[9]	validation_0-rmse:1572.29508
[10]	validation_0-rmse:1557.71894
[11]	validation_0-rmse:1543.29914
[12]	validation_0-rmse:1529.03426
[13]	validation_0-rmse:1514.92273
[14]	validation_0-rmse:1500.96322
[15]	validation_0-rmse:1487.15421
[16]	validation_0-rmse:1473.49445
[17]	validation_0-rmse:1459.98230
[18]	validation_0-rmse:1446.61655
[19]	validation_0-rmse:1433.39562
[20]	validation_0-rmse:1420.31837
[21]	validation_0-rmse:1407.38334
[22]	validation_0-rmse:1394.58920
[23]	validation_0-rmse:1381.93461
[24]	validation_0-rmse:1369.41834
[25]	validation_0-rmse:1357.03898
[26]	validation_0-rmse:1344.79531
[27]	validation_0-rmse:1332.68611
[28]	validation_0-rmse:1320.70994
[29]	validation_0-rmse:1

...

C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1706.60621
[1]	validation_0-rmse:1690.42170
[2]	validation_0-rmse:1674.40709
[3]	validation_0-rmse:1658.56082
[4]	validation_0-rmse:1642.88124
[5]	validation_0-rmse:1627.36675
[6]	validation_0-rmse:1612.01581
[7]	validation_0-rmse:1596.82681
[8]	validation_0-rmse:1581.79823
[9]	validation_0-rmse:1566.92843
[10]	validation_0-rmse:1552.21597
[11]	validation_0-rmse:1537.65935
[12]	validation_0-rmse:1523.25702
[13]	validation_0-rmse:1509.00763
[14]	validation_0-rmse:1494.90955
[15]	validation_0-rmse:1480.96155
[16]	validation_0-rmse:1467.16200
[17]	validation_0-rmse:1453.50960
[18]	validation_0-rmse:1440.00287
[19]	validation_0-rmse:1426.64053
[20]	validation_0-rmse:1413.42113
[21]	validation_0-rmse:1400.34337
[22]	validation_0-rmse:1387.39859
[23]	validation_0-rmse:1374.59278
[24]	validation_0-rmse:1361.92474
[25]	validation_0-rmse:1349.40160
[26]	validation_0-rmse:1337.00467
[27]	validation_0-rmse:1324.74167
[28]	validation_0-rmse:1312.61294
[29]	validation_0-rmse:1

C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

[0]	validation_0-rmse:1716.10155
[1]	validation_0-rmse:1699.83608
[2]	validation_0-rmse:1683.74159
[3]	validation_0-rmse:1667.81642
[4]	validation_0-rmse:1652.05900
[5]	validation_0-rmse:1636.46763
[6]	validation_0-rmse:1621.04084
[7]	validation_0-rmse:1605.77479
[8]	validation_0-rmse:1590.67229
[9]	validation_0-rmse:1575.72739
[10]	validation_0-rmse:1560.94301
[11]	validation_0-rmse:1546.31322
[12]	validation_0-rmse:1531.83879
[13]	validation_0-rmse:1517.52027
[14]	validation_0-rmse:1503.35212
[15]	validation_0-rmse:1489.33448
[16]	validation_0-rmse:1475.46654
[17]	validation_0-rmse:1461.74625
[18]	validation_0-rmse:1448.17283
[19]	validation_0-rmse:1434.74424
[20]	validation_0-rmse:1421.45977
[21]	validation_0-rmse:1408.31740
[22]	validation_0-rmse:1395.31642
[23]	validation_0-rmse:1382.45486
[24]	validation_0-rmse:1369.73204
[25]	validation_0-rmse:1357.14596
[26]	validation_0-rmse:1344.69608
[27]	validation_0-rmse:1332.38032
[28]	validation_0-rmse:1320.19209
[29]	validation_0-rmse:1

C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_10672\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

# Melhor Métrica

In [23]:
df = pd.read_csv('metrics.csv')
df['Valor'] = df['Valor'].apply(lambda x: float(x))

In [18]:
min_a = df.loc[df['Métrica'] == 'RRSE','Valor'].min()
a = df.loc[df['Valor'] == min_a]
a
#a.to_csv('melhor_rrse.csv',index=False)

In [19]:
min_b = df.loc[df['Métrica'] == 'MAPE','Valor'].min()
b = df.loc[df['Valor'] == min_b]
b
#b.to_csv('melhor_mape.csv',index=False)